In [0]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Concatenate, Lambda, Dense, Activation, Input, AveragePooling2D, Flatten, Dropout
from tensorflow.nn import local_response_normalization
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import sklearn 

*Today i will implement the InceptionNet version 1 based on this wonderful and  specific architecture from medium* 


![alt text](https://mk0learnfx6ydjq1jij.kinstacdn.com/wp-content/uploads/2019/09/1_53uKkbeyzJcdo8PE5TQqqw-1-1024x700.png)

Mong la khong fail :v

In [0]:
def LocResNorm(X):
  X = local_response_normalization(X)
  return X
def Inception_module(X, filters):
  X_conv1 = X
  X_conv3 = X
  X_conv5 = X
  X_pool = X
  F1_conv1_1, F1_conv1_3, F2_conv3, F1_conv1_5, F2_conv5, F2_conv1 = filters
  X_conv1 = Conv2D(F1_conv1_1, kernel_size = (1,1), padding = "same", kernel_initializer = glorot_uniform(seed = 0))(X_conv1)
  X_conv1 = Activation("relu")(X_conv1)
  X_conv3 = Conv2D(F1_conv1_3, kernel_size=(1,1), padding = "same", kernel_initializer= glorot_uniform(seed = 0))(X_conv3)
  X_conv3 = Activation("relu")(X_conv3)
  X_conv3 = Conv2D(F2_conv3, kernel_size= (3,3), padding = "same", kernel_initializer = glorot_uniform(seed = 0))(X_conv3)
  X_conv3 = Activation("relu")(X_conv3)
  X_conv5 = Conv2D(F1_conv1_5, kernel_size=(1,1), padding = "same", kernel_initializer= glorot_uniform(seed = 0))(X_conv5)
  X_conv5 = Activation("relu")(X_conv5)
  X_conv5 = Conv2D(F2_conv5, kernel_size= (5,5), padding = "same", kernel_initializer = glorot_uniform(seed = 0))(X_conv5)
  X_conv5 = Activation("relu")(X_conv5)
  X_pool = MaxPooling2D(pool_size=(3,3), padding = "same", strides = (1,1))(X_pool)
  X_pool = Activation("relu")(X_pool)
  X_pool = Conv2D(F2_conv1, kernel_size= (1,1), padding = "same", kernel_initializer = glorot_uniform(seed = 0))(X_pool)
  X_pool = Activation("relu")(X_pool)
  X = Concatenate(axis = 3)([X_conv1, X_conv3, X_conv5, X_pool])
  return X
def Stem_module(X, filters):
  F_conv7, F_conv1, F_conv3 = filters
  X = Conv2D(F_conv7, kernel_size = (7,7), strides = (2,2), kernel_initializer= glorot_uniform(seed = 0), padding = "same")(X)
  X = Activation("relu")(X)
  X = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding= "same")(X)
  X = LocResNorm(X)
  X = Conv2D(F_conv1, kernel_size=(1,1), padding = "valid", strides = (1,1), kernel_initializer=glorot_uniform(seed = 0))(X)
  X = Activation("relu")(X)
  X = Conv2D(F_conv3, kernel_size=(3,3), kernel_initializer = glorot_uniform(seed = 0), strides = (1,1), padding = "same")(X)
  X = Activation("relu")(X)
  X = LocResNorm(X)
  X = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = "valid")(X)
  return X
def Auxiliary_module(X, filter, classes):
  X = AveragePooling2D(pool_size = (5,5), strides = (3,3), padding = "valid")(X)
  X = Conv2D(filter, kernel_size = (1,1), strides = (1,1), padding = "same", kernel_initializer = glorot_uniform(seed = 0))(X)
  X = Flatten()(X)
  X = Dense(512, activation = "relu", kernel_initializer = glorot_uniform(seed = 0))(X)
  X = Dense(classes, activation = "softmax", kernel_initializer = glorot_uniform(seed = 0))(X)
  return X

In [0]:
def GoogleNet_v1(input_shape, classes):
  X_input = Input(input_shape)
  X = Stem_module(X_input, [64,64,192])
  X = Inception_module(X, [64, 96, 128, 16, 32, 32])
  X = Inception_module(X, [128, 128, 192, 32, 96, 64])
  X = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = "same")(X)
  X = Inception_module(X, [192, 96, 208, 16, 48, 64])
  X_aux1 = X
  softmax_0 = Auxiliary_module(X_aux1, 128, 10)
  X = Inception_module(X, [160, 112, 224, 24, 64, 64])
  X = Inception_module(X, [128, 128, 256, 24, 64, 64])
  X = Inception_module(X, [112, 144, 288, 32, 64, 64])
  X_aux2 = X
  softmax_1 = Auxiliary_module(X_aux2, 128, 10)
  X = Inception_module(X, [256, 160, 320, 32, 128, 128])
  X = MaxPooling2D(pool_size=(3,3), strides = (2,2), padding = "same")(X)
  X = Inception_module(X, [256, 160, 320, 32, 128, 128])
  X = Inception_module(X, [384, 192, 384, 48, 128, 128])
  X = AveragePooling2D(pool_size = (7,7), strides = (1,1), padding = "valid")(X)
  X = Flatten()(X)
  X = Dropout(0.4)(X)
  X = Dense(512, activation = "relu", kernel_initializer = glorot_uniform(seed = 0))(X)
  softmax_2 = Dense(classes, activation = "softmax", kernel_initializer = glorot_uniform(seed = 0))(X)
  model = Model(inputs = X_input, outputs = [softmax_0, softmax_1, softmax_2], name = "GoogleNet_v1")
  return model

In [0]:
def loss_func(y_true, soft_max_list):
  softmax_0, softmax_1, softmax_2 = soft_max_list
  Comp_0 = tf.keras.losses.categorical_crossentropy(y_true, softmax_0)
  Comp_1 = tf.keras.losses.categorical_crossentropy(y_true, softmax_1)
  Comp_2 = tf.keras.losses.categorical_crossentropy(y_true, softmax_2)
  return Comp_2 + 0.3*Comp_1 + 0.3*Comp_0 

In [0]:
model = GoogleNet_v1((224, 224, 3), 10)

In [0]:
model.compile(optimizer= "adam", loss = loss_func, metrics = ["accuracy"])